In [13]:
import requests
import time
import pandas as pd 
import numpy as np 

# textrank
from konlpy.tag import Kkma
from konlpy.tag import Twitter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import normalize

from selenium import webdriver 
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
import re

def briefing_craw():
    # 1. 코로나바이러스감염증-19 최근 게시된 공지사항 크롤링
    url = 'http://ncov.mohw.go.kr/tcmBoardList.do?brdId=&brdGubun=&dataGubun=&ncvContSeq=&contSeq=&board_id=140&gubun='

    options = webdriver.ChromeOptions()
    options.add_argument('headless')
    options.add_argument('window-size=1920x1080')
    options.add_argument("disable-gpu")

    browser = webdriver.Chrome('chromedriver', chrome_options=options) # 같은 디렉토리 내에 chtromdriver.exe
    browser.implicitly_wait(1)

    browser.get(f'{url}')
    browser.implicitly_wait(1)

    browser.find_element_by_css_selector("""#content > div > div.board_list > table > tbody > tr:nth-child(1) > td.ta_l > a""").click()
    texts = browser.find_element_by_css_selector("""#content > div > div.board_view > div.bv_content""").text
    browser.close()
    
    texts = re.sub('[□○※【】]', '', texts)
    texts = texts.replace('▲', '')
    texts = texts.replace('▸', ', ')
    
    # 2. textrank
    class SentenceTokenizer(object):
    
        def __init__(self):
            self.kkma = Kkma()
            self.twitter = Twitter()
            self.stopwords = ['뉴스','연합', '자료사진','서울연합','중인' ,'만큼', '마찬가지', '꼬집었', "연합뉴스", "데일리", "동아일보", "중앙일보", "조선일보", "기자"
            ,"아", "휴", "아이구", "아이쿠", "아이고", "어", "나", "우리", "저희", "따라", "의해", "을", "를", "에", "의", "가", "▲",
            "None"]

            
        # Kkma를 이용하여 문장단위로 나눈 후 setences를 return
        def url2sentences(self, url):
            article = Article(url, language='ko')
            article.download()
            article.parse()
            sentences = self.kkma.sentences(article.text)

            for idx in range(0, len(sentences)):
                if len(sentences[idx]) <= 10:
                    sentences[idx-1] += (' ' + sentences[idx])
                    sentences[idx] = ''

            return sentences

        def text2sentences(self, text):
            sentences = self.kkma.sentences(text)
            for idx in range(0, len(sentences)):
                if len(sentences[idx]) <= 10:
                    sentences[idx-1] += (' ' + sentences[idx])
                    sentences[idx] = ''

            return sentences

        # sentences를 받아 twitter.nouns()를 이용하여 명사를 추출하고 nouns를 return
        def get_nouns(self, sentences):
            nouns = []
            for sentence in sentences:
                if sentence is not '':
                    nouns.append(' '.join([noun for noun in self.twitter.nouns(str(sentence))
                                            if noun not in self.stopwords and len(noun) > 1]))

            return nouns

        
    # TF-IDF 모델, 그래프 생성과정을 GraphMatrix 클래스로 구현
    class GraphMatrix(object):
        def __init__(self):
            self.tfidf = TfidfVectorizer()
            self.cnt_vec = CountVectorizer()
            self.graph_sentence = []

        def build_sent_graph(self, sentence):
            tfidf_mat = self.tfidf.fit_transform(sentence).toarray()
            self.graph_sentence = np.dot(tfidf_mat, tfidf_mat.T)
            return  self.graph_sentence

        def build_words_graph(self, sentence):
            cnt_vec_mat = normalize(self.cnt_vec.fit_transform(sentence).toarray().astype(float), axis=0)
            vocab = self.cnt_vec.vocabulary_
            return np.dot(cnt_vec_mat.T, cnt_vec_mat), {vocab[word] : word for word in vocab}
        
        
    # textrank -> 가중치그래프를 이용하여 적용 -> 높은 순으로 정렬 -> 요약할 단어 갯수만큼 출력

    class Rank(object):
        def get_ranks(self, graph, d=0.85): # d = damping factor
            A = graph
            matrix_size = A.shape[0]
            for id in range(matrix_size):
                A[id, id] = 0 # diagonal 부분을 0으로
                link_sum = np.sum(A[:,id]) # A[:, id] = A[:][id]
                if link_sum != 0:
                    A[:, id] /= link_sum
                A[:, id] *= -d
                A[id, id] = 1

            B = (1-d) * np.ones((matrix_size, 1))
            ranks = np.linalg.solve(A, B) # 연립방정식 Ax = b
            return {idx: r[0] for idx, r in enumerate(ranks)}
        
        
    class TextRank(object):
        def __init__(self, text):
            self.sent_tokenize = SentenceTokenizer()

            if text[:5] in ('http:', 'https'):
                self.sentences = self.sent_tokenize.url2sentences(text)
            else:
                self.sentences = self.sent_tokenize.text2sentences(text)

            self.nouns = self.sent_tokenize.get_nouns(self.sentences)

            self.graph_matrix = GraphMatrix()
            self.sent_graph = self.graph_matrix.build_sent_graph(self.nouns)
            self.words_graph, self.idx2word = self.graph_matrix.build_words_graph(self.nouns)

            self.rank = Rank()
            self.sent_rank_idx = self.rank.get_ranks(self.sent_graph)
            self.sorted_sent_rank_idx = sorted(self.sent_rank_idx, key=lambda k: self.sent_rank_idx[k], reverse=True)

            self.word_rank_idx = self.rank.get_ranks(self.words_graph)
            self.sorted_word_rank_idx = sorted(self.word_rank_idx, key=lambda k: self.word_rank_idx[k], reverse=True)

        # 요약문장 3문장
        def summarize(self, sent_num=3):
            summary = []
            index=[]
            for idx in self.sorted_sent_rank_idx[:sent_num]:
                index.append(idx)
            index.sort()
            for idx in index:
                summary.append(self.sentences[idx])

            return summary

        # 키워드 10개
        def keywords(self, word_num=5):
            rank = Rank()    
            rank_idx = rank.get_ranks(self.words_graph)
            sorted_rank_idx = sorted(rank_idx, key=lambda k: rank_idx[k], reverse=True)

            keywords = []
            index=[]
            for idx in sorted_rank_idx[:word_num]:
                index.append(idx)

            #index.sort()
            for idx in index:
                keywords.append(self.idx2word[idx])

            return keywords

    url = texts
    textrank = TextRank(url)
    for row in textrank.summarize(3):
        print(row)
        print()
    
    print('keywords :',textrank.keywords())
    
    
briefing_craw()

c:\users\playdata\.conda\envs\env-01\lib\site-packages\ipykernel_launcher.py:27: DeprecationWarning: use options instead of chrome_options
c:\users\playdata\.conda\envs\env-01\lib\site-packages\konlpy\tag\_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


- 코로나 19 극복 방안 논의 위한 보건복지 부 제 1 차관과 콜롬비아 보건 부 차관 면담 (11.23) - 코로나 19 대응을 위해 보건복지 부 양성일 제 1 차관과 루이즈 알렉산 더 모스코 소 (Luis Alexander Moscoso) 콜롬비아 보건 부 차관 간의 면담이 11월 23일( 월) 17시 서울 콘래드 호텔에서 열렸다.

보건복지 부 양성일 제 1차 관은 면담에서 한국의 코로나 19 상황 및 대응 방식과 백신 확보 전략을 설명하면서, 전 세계적인 보건위기 상황에서 한국과 콜롬비아 간 보건‧ 의료 연대가 그 어느 때보다 절실한 시기라고 강조하였다.



keywords : ['콜롬비아', '보건', '코로나', '협력', '대응']
